# Postgres Documentation

In [189]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [190]:
%sql postgresql://postgres@psql:5432/postgres

'Connected: postgres@postgres'

## Data structure and authors
In our Dataset we have all authors set to 'Books' as a author string row, rather than having them in their own table that's referenced by 'Books' table with a integer id, this would have been proper normalization, and we would have corrected this weren't it for time constraints.

so looking through our data to find authors we have to use something akin to this query.

In [411]:
%sql select * from books where author like 'William Shakespeare';

21 rows affected.


id,title,author
291,Shakespeare's Sonnets,William Shakespeare
321,Venus and Adonis,William Shakespeare
887,King Richard III,William Shakespeare
972,Romeo and Juliet,William Shakespeare
1030,Much Ado About Nothing,William Shakespeare
1057,As You Like It,William Shakespeare
2297,The Merchant of Venice [liberally edited by Charles Kean],William Shakespeare
2525,A Fairy Tale in Two Acts Taken from Shakespeare (1763),William Shakespeare
4693,Othello· the Moor of Venice,William Shakespeare
4764,The Winter's Tale,William Shakespeare


In [276]:
%sql EXPLAIN ANALYZE select * from books where author like 'William Shakespeare';

5 rows affected.


QUERY PLAN
Seq Scan on books (cost=0.00..845.26 rows=2 width=50) (actual time=0.024..2.155 rows=21 loops=1)
Filter: ((author)::text ~~ 'William Shakespeare'::text)
Rows Removed by Filter: 37200
Planning time: 0.033 ms
Execution time: 2.165 ms


Problem is as we stated earlier, our Books.author contains multiple authors in some cases, so we have to use Regular expression to find all cases.

In [257]:
%sql select * from books where author like '%William Shakespeare%';

25 rows affected.


id,title,author
291,Shakespeare's Sonnets,William Shakespeare
321,Venus and Adonis,William Shakespeare
887,King Richard III,William Shakespeare
972,Romeo and Juliet,William Shakespeare
1030,Much Ado About Nothing,William Shakespeare
1057,As You Like It,William Shakespeare
2297,The Merchant of Venice [liberally edited by Charles Kean],William Shakespeare
2525,A Fairy Tale in Two Acts Taken from Shakespeare (1763),William Shakespeare
4566,The Merry Wives of Windsor,William Shakespeare [Craig· Oxford edition]
4693,Othello· the Moor of Venice,William Shakespeare


In [292]:
%sql EXPLAIN ANALYSE select * from books where author like '%William Shakespeare%';

5 rows affected.


QUERY PLAN
Seq Scan on books (cost=0.00..845.26 rows=3 width=50) (actual time=0.030..3.041 rows=25 loops=1)
Filter: ((author)::text ~~ '%William Shakespeare%'::text)
Rows Removed by Filter: 37196
Planning time: 0.037 ms
Execution time: 3.052 ms


It's interesting to note that the more accurate query take 50% longer or 1 milisecond longer to complete, 1 milisecond isn't much but 50% is a more notable value in this instance, but we don't have any indexes on author, and we can't really apply any as we use wildcard syntax, which is problematic given a btree'slookup speed by big O notation is: ![](https://render.githubusercontent.com/render/math?math=O%28log%28n%29%29&mode=inline) 

as a example if we choose to use a btree.

In [326]:
%sql CREATE INDEX book_author_index ON books USING btree("author");

Done.


[]

In [345]:
%sql EXPLAIN ANALYZE select * from books where author like 'William Shakespeare';

7 rows affected.


QUERY PLAN
Bitmap Heap Scan on books (cost=4.31..11.90 rows=2 width=50) (actual time=0.016..0.027 rows=21 loops=1)
Filter: ((author)::text ~~ 'William Shakespeare'::text)
Heap Blocks: exact=13
-> Bitmap Index Scan on book_author_index (cost=0.00..4.30 rows=2 width=0) (actual time=0.011..0.011 rows=21 loops=1)
Index Cond: ((author)::text = 'William Shakespeare'::text)
Planning time: 0.048 ms
Execution time: 0.039 ms


Planning suddenly takes longer than execution time, however the execution time is 55 times FASTER, that's 5500% higher speed, and the total speed is 25 times faster than the without indexing the author.

However since we use wildcard syntax...

In [328]:
%sql EXPLAIN ANALYSE select * from books where author like '%William Shakespeare%';

5 rows affected.


QUERY PLAN
Seq Scan on books (cost=0.00..845.26 rows=3 width=50) (actual time=0.031..3.051 rows=25 loops=1)
Filter: ((author)::text ~~ '%William Shakespeare%'::text)
Rows Removed by Filter: 37196
Planning time: 0.042 ms
Execution time: 3.063 ms


then the actual accurate query doesn't gain any speed, and we end up wasting ram allocation on a index that's practically useless, it must be stressed again as we documented earlier in this report, this datastructure is the result of time constraint, and is in no way optimally normalized.

## Spartial data
We could explain many other queries, but the best showcase of the data in action comes from the fouth query stating:   
`Given a geolocation, your application lists all books mentioning a city in vicinity of the given geolocation.`   
this has been a interesting one, as we need to get city data and reversely find all books that have mentioned that city in particular, the biggest problem have been using different earth cordinates solutions to find the related cities, and mirroring the queries with other databases to uniform the results.

### Geometric Functions and Operators
doing our course we have been introduced to postgres own spartial data operators, that allow for easy spartial related queries, the issue we found however, relates to the values and and how earth's cordinates doesn't fit with a flat plain as earth is oblate spheroid in shape.   
[Postgres documentation on geometric functions and operators](https://www.postgresql.org/docs/current/static/functions-geometry.html)

### Haversine Formula
another way to do spartial data queries was introduced to us in the form of the haversine formular, and we used this through the majority of our development circle, however we later learned from testing, that the solution we used calculated in miles instead of kilometers, and we choose to find another way to correctly query the data rather than tinker with the current solution.


### Postgres Extentions
doing our search for new solutions to correctly handle our spartial data queries we found the `earthdistance` module for postgres [here](https://www.postgresql.org/docs/8.3/static/earthdistance.html), it calculated in meters however it assumes the earth is perfectly spherical, which isn't correct and points towards [PostGIS](http://www.postgis.org/) as a solution for more accurate calculations, however we choose to go with earthdistance as we had already run through numerous calculation solutions, and were pressed for time.

In [52]:
%sql create extension earthdistance cascade;

Done.


[]

### Indexes
as spartial data is our main comparison tool, it seemed to be correct to use Rtrees as it primarily uses rectangles to store spartial data, since our solution contains even formed cordinates, it should be the best choice for this task, however as we where reading [postgres documentation on indexes](https://www.postgresql.org/docs/8.1/static/indexes-types.html) we learned this was not the case.

```
Note: Testing has shown PostgreSQL's hash indexes to perform no better than B-tree indexes, and the index size and build time for hash indexes is much worse. Furthermore, hash index operations are not presently WAL-logged, so hash indexes may need to be rebuilt with REINDEX after a database crash. For these reasons, hash index use is presently discouraged.

Similarly, R-tree indexes do not seem to have any performance advantages compared to the equivalent operations of GiST indexes. Like hash indexes, they are not WAL-logged and may need reindexing after a database crash.

While the problems with hash indexes may be fixed eventually, it is likely that the R-tree index type will be retired in a future release. Users are encouraged to migrate applications that use R-tree indexes to GiST indexes.
```

so as adviced we use Generalised search tree index instead.

In [434]:
%sql CREATE INDEX latitude_longitude_index ON cities USING gist (ll_to_earth(latitude, longitude));

Done.


[]

### Performance
to save space the final results will be cut down and centered around this query
```
%%sql
select cities.name, cities.latitude, cities.longitude, books.id, books.title from cities
join mentions on (cities.id = mentions.cityid)
join books on (mentions.bookid = books.id)
where earth_box(ll_to_earth(52.38, 11.47), 50000) @> ll_to_earth(latitude, longitude)
and earth_distance(ll_to_earth(52.38, 11.47), ll_to_earth(latitude, longitude)) < 50000;
```
to follow the logic we choose all cities and books using equi join on the Forign keys of Mentions, this cuts away any city that isn't mentioned by any book. Then filter using a rectangle box to cut down, so the radial filter have less to run though, the reason for this is the index have a faster use of rectangle, without the index and using only `earth_distance(ll_to_earth(52.38, 11.47), ll_to_earth(latitude, longitude)) < 50000` the result is this:   

without ![](https://i.gyazo.com/0e55fbb6c3f484d97d9c80cc26fce255.png)
with ![](https://i.gyazo.com/301e633bd19215a634b826cdd728b787.png)
so clearly without the index our choice of including both doesn't work, however the rectangle filter works very well with the index which can clearly be seen in these results:

without ![](https://i.gyazo.com/1ce99f6a47f4044a4e06650396467668.png)
with ![](https://i.gyazo.com/ac0d2dd074c442bf59ea4df83e877cfd.png)